In [1]:
from bs4 import BeautifulSoup
from fnmatch import fnmatch
from pathlib import Path
from shutil import copyfile

import mysql.connector
import os
import re

In [2]:
# GENERAL CONFIGS

# == Database config ==
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root"
)

cursor = conn.cursor()

DB_NAME = "blp_dataset"
TABLE_NAME = "products_tmp"


# == Files config ==
MAIN_FOLDER = r"D:\dnmarchives\novo\pandora"

In [3]:
# Creating environment
cursor.execute("CREATE DATABASE IF NOT EXISTS {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
cursor.execute("USE {}".format(DB_NAME))

table_schema = (
    "CREATE TABLE IF NOT EXISTS products_tmp ("
    "  id INT(11) AUTO_INCREMENT PRIMARY KEY NOT NULL,"
    "  name TEXT NOT NULL,"
    "  description TEXT,"
    "  price VARCHAR(50),"
    "  market_name VARCHAR(255),"
    "  seller_name TEXT,"
    "  has_image VARCHAR(25) NOT NULL DEFAULT 'has_image',"
    "  illegal ENUM('t', 'f') NOT NULL DEFAULT 't'"
    ") ENGINE=InnoDB")

cursor.execute(table_schema)

In [4]:
original_images = []

folders = [x for x in Path(MAIN_FOLDER).iterdir() if x.is_dir()]

for fo in folders:
    folder = Path(f"{fo}\item")
    folder_image = Path(f"{fo}")

    if folder.exists():
        for file in os.listdir(folder):
            page_content = BeautifulSoup(open(Path(f"{folder}\{file}"), encoding='utf8').read())

            ## GET DESCRIPTION
            description = ""
            # find description main container
            raw_desc = page_content.find("pre")

            if raw_desc is not None:
                # remove html content, whitespaces and line breaks
                raw_description = raw_desc.text
                description = re.sub(r'[\t\r\n]', ' ', raw_description)
                description = re.sub(' +',' ', description).lstrip()

            ## GET TITLE
            title = ""
            raw_title = page_content.find("th", attrs={"colspan": "2"})

            if raw_title is not None:
                for strong in raw_title.find_all("strong", recursive=False):
                    if strong is not None:
                        text_title = strong.text

                        if text_title is not None:
                            title = re.sub(' +',' ', text_title).lstrip()

            ## GET SELLER
            seller_name = ""

            seller_a = page_content.find_all("a", href=lambda value: value and value.startswith("/profile/"))

            if seller_a is not None:
                for seller_raw in seller_a:
                    seller_name = seller_raw.text

            if description != "" or title != "":
                # Insert product into database
                query = "INSERT INTO products_tmp (name, description, market_name, seller_name, illegal) VALUES (%s, %s, %s, %s, %s)"
                values = (title, description, "Pandora", seller_name, "t")

                cursor.execute(query, values)
                conn.commit()

                product_id = cursor.lastrowid

                ## GET IMAGE
                raw_name = page_content.find("img")['src']

                if raw_name == "img/no-image.png":
                    query = "UPDATE products SET has_image = %s WHERE id = %s"
                    cursor.execute(query, ("no_image", product_id))
                    conn.commit()
                else:
                    if raw_name in original_images:
                        query = "UPDATE products SET has_image = %s WHERE id = %s"
                        cursor.execute(query, ("repeated", product_id))
                        conn.commit()
                    else:
                        original_images.append(raw_name)

                        url_image = Path(f"{folder_image}\{raw_name}")

                        # copy images to new directory, renaming with product id
                        if url_image.exists():
                            dst_dir = Path(f"D:\images\pandora\{product_id}.jpg")
                            copyfile(url_image, dst_dir)
                        else:
                            query = "UPDATE products SET has_image = %s WHERE id = %s"
                            cursor.execute(query, ("no_image", product_id))
                            conn.commit()